In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import pandas as pd
import numpy as np

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [37]:
model_path = './models/tabnet_multy_30.pth'

In [38]:
model = torch.load(model_path, map_location=device, weights_only=False)
model.eval()
model.to(device)

TabNet(
  (initial_bn): BatchNorm1d(77, eps=1e-05, momentum=None, affine=True, track_running_stats=True)
  (initial_splitter): FeatureTransformer(
    (shared): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=77, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=1e-05, momentum=None, affine=True, track_running_stats=True)
        (2): GLU(dim=-1)
        (3): Dropout(p=0.1, inplace=False)
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=1e-05, momentum=None, affine=True, track_running_stats=True)
        (2): GLU(dim=-1)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
    (independent): ModuleList(
      (0-1): 2 x Sequential(
        (0): Linear(in_features=64, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=1e-05, momentum=None, affine=True, track_running_stats=True)
        (2): GLU(dim=-1)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
 

In [39]:
DATASET_PATH = 'datasets/data/different_features/traffic_features.csv'
data = pd.read_csv(DATASET_PATH, delimiter=',')

X = data.drop('Label', axis=1)
y = LabelEncoder().fit_transform(data['Label'])
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Разделение на тренировочный и тестовый наборы (или используйте существующий test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Преобразование в тензоры
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

# Создание DataLoader
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)  # Используйте batch_size из params

# Оценка точности
y_pred = []
y_true = []

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output, _, _ = model(data)  # Предполагаем, что forward возвращает (output, M, M_loss)
        pred = output.argmax(dim=1)
        y_pred.extend(pred.cpu().numpy())
        y_true.extend(target.cpu().numpy())

# Вычисление метрик
accuracy = accuracy_score(y_true, y_pred) * 100
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)
conf_matrix = confusion_matrix(y_true, y_pred)

# Вывод результатов
print(f"Точность (Accuracy): {accuracy:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")
print("\nМатрица ошибок:")
print(conf_matrix)

Точность (Accuracy): 89.84%
Precision: 0.90
Recall: 0.90
F1-Score: 0.90

Матрица ошибок:
[[21  1  1  0  0  0  1]
 [ 1 23  0  0  0  0  0]
 [ 1  2 81  3  0  0  0]
 [ 1  0  0 31  0  0  2]
 [ 0  1  0  1 24  1  1]
 [ 0  0  0  0  0 33  1]
 [ 0  0  2  2  4  0 17]]
